In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Configurações gerais
plt.rcParams.update({'figure.max_open_warning': 0})

# Diretório para salvar as figuras
output_dir = 'figuras_lstm_puro'
os.makedirs(output_dir, exist_ok=True)

# 1. Leitura do dataset a partir do arquivo CSV
df = pd.read_csv('pca_reduced_data.csv')
Q_reduced = df.iloc[:, ::2]  # Seleciona apenas as colunas necessárias (colunas pares)
Q_reduced = Q_reduced.T

# Renomear as colunas de Q_reduced
Q_reduced.columns = [f'feature_{i+1}' for i in range(Q_reduced.shape[1])]

time_steps = np.arange(Q_reduced.shape[0])

# 4. Definição da janela deslizante e número de características
window_size = 25  # Tamanho da janela deslizante para criar sequências temporais
n_features = Q_reduced.shape[1]  # Número de características (features)

# 5. Normalização dos dados
scaler = MinMaxScaler(feature_range=(-1, 1))
Q_reduced_scaled = scaler.fit_transform(Q_reduced)

# 7. Preparação das sequências
def create_sequences(data, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i + window_size])
        labels.append(data[i + window_size])
    return np.array(sequences), np.array(labels)

# Criar sequências a partir dos dados normalizados
X, y = create_sequences(Q_reduced_scaled, window_size)
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# 8. Divisão dos dados em conjuntos de treinamento e teste com continuidade temporal
treino = 0.75
split_index = int(len(X) * treino)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# 9. Definição do modelo LSTM puro com camadas adicionais e dropout
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(window_size, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(n_features))

# 10. Compilação do modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Definindo EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 11. Treinamento do modelo
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32,
                    callbacks=[early_stopping], verbose=1)

# 12. Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# 13. Reverter a normalização das previsões e valores reais
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# 14. Calcular métricas de erro
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}')

# Função para calcular NMSE por característica
def calculate_normalized_mse(actual, predicted):
    num_features = actual.shape[1]
    nmse = np.zeros(num_features)
    for c in range(num_features):
        mse = mean_squared_error(actual[:, c], predicted[:, c])
        variance = np.var(actual[:, c])
        if variance != 0:
            nmse[c] = mse / variance
        else:
            nmse[c] = np.nan
    return nmse

# Calcular NMSE para cada característica
nmse = calculate_normalized_mse(y_test_inv, y_pred_inv)
average_nmse = np.nanmean(nmse)
print(f"Média do NMSE: {average_nmse:.4f}")

# 15. Comparar previsões com valores reais em subplots
num_features_to_plot = 4  # Número de características a serem plotadas
features_to_plot = np.random.choice(n_features, num_features_to_plot, replace=False)

fig, axs = plt.subplots(num_features_to_plot, 1, figsize=(12, 3 * num_features_to_plot), dpi=400)
for idx, feature_idx in enumerate(features_to_plot):
    axs[idx].plot(y_test_inv[:, feature_idx], label=f'Valor Real (Feature {feature_idx+1})')
    axs[idx].plot(y_pred_inv[:, feature_idx], label=f'Previsão (Feature {feature_idx+1})', linestyle='--')
    axs[idx].set_title(f'Feature {feature_idx+1} - Previsão vs Valor Real')
    axs[idx].set_xlabel('Amostra')
    axs[idx].set_ylabel('Valor')
    axs[idx].legend()
    axs[idx].grid(True)
    # Rotacionar os rótulos do eixo x
    axs[idx].tick_params(axis='x', rotation=45)
    # Reduzir o número de ticks no eixo x
    axs[idx].xaxis.set_major_locator(plt.MaxNLocator(10))
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'comparacao_features.png'), dpi=400)
plt.close()

# 16. Plotar NMSE por característica
features = np.arange(1, n_features + 1)
plt.figure(figsize=(12, 6), dpi=400)
plt.bar(features, nmse, width=0.6, color='blue')
plt.xlabel('Índice da Característica')
plt.ylabel('NMSE')
plt.title('NMSE por Característica')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'nmse_por_caracteristica.png'), dpi=400)
plt.close()

# 17. Plotar valores de perda de treinamento e validação
plt.figure(figsize=(12, 6), dpi=400)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Função de Perda ao Longo das Épocas')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)
# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'funcao_perda.png'), dpi=400)
plt.close()


X shape: (375, 25, 10)
y shape: (375, 10)


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 259ms/step - loss: 0.4469 - val_loss: 0.3244
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.3073 - val_loss: 0.1874
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.1921 - val_loss: 0.1051
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.1159 - val_loss: 0.0541
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0721 - val_loss: 0.0278
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.0507 - val_loss: 0.0165
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0413 - val_loss: 0.0096
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0318 - val_loss: 0.0068
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 0.0307 - val_loss: 0.0070
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0289 - val_loss: 0.0047
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0271 - val_loss: 0.0042
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0247 - val_